In [1]:
# Cell 1 — Imports & config
import pandas as pd
from pathlib import Path

# === CHOOSE YOUR CSV FILE HERE ===
# Options: 'menu.csv', 'orders.csv', 'registration.csv', 'staff.csv'
csv_file_name = 'menu.csv'
# =================================

CSV_PATH = Path(f'../doc/{csv_file_name}')
OUTPUT_SQL = Path(f'{CSV_PATH.stem}_inserts.sql')

# Helper: SQL quote and NULL handling
def sql_quote(val):
    if pd.isna(val):
        return "NULL"
    s = str(val)
    if s.strip() == "":
        return "NULL"
    s = s.replace("'", "''")
    return f"'{s}'"

In [2]:
# Cell 2 — Load CSV and display headers
df = pd.read_csv(CSV_PATH)

print(f"Headers for {CSV_PATH.name}:")
for col in df.columns:
    print(f'- {col}')

print('First 5 rows:')
df.head()

Headers for menu.csv:
- Item
- Price
- Cuisine
First 5 rows:


,Item,Price,Cuisine
0,Rendang,4.0,Indonesian
1,Ayam Balado,4.0,Indonesian
2,Gudeg,3.0,Indonesian
3,Rinderrouladen,3.5,German
4,Sauerbraten,4.0,German


In [ ]:
# Cell 3 — Select which columns to include in the INSERT statements
# === CHOOSE YOUR COLUMNS HERE ===
# You can now select from the detected columns or use all of them
selected_columns = list(df.columns)  # This will use all detected columns
# Or uncomment below to manually select from detected columns
# selected_columns = ["Date", "Time", "Phone", "Firstname", "Lastname"]
# =================================

# Verify that the selected columns exist in the DataFrame
missing_cols = [col for col in selected_columns if col not in df.columns]
if missing_cols:
    raise ValueError(f"The following columns were not found in the CSV: {missing_cols}")

df_selected = df[selected_columns]
print(f"Selected columns for INSERT: {', '.join(df_selected.columns)}")
df_selected.head()

ValueError: The following columns were not found in the CSV: ['Date', 'Time', 'Phone', 'Firstname', 'Lastname']

In [3]:
# Cell 4 — Generate and preview PostgreSQL INSERT statements
inserts = []
table_name = CSV_PATH.stem

for _, row in df_selected.iterrows():
    values = ', '.join([sql_quote(row[col]) for col in df_selected.columns])
    stmt = f"INSERT INTO {table_name} ({', '.join(df_selected.columns)}) VALUES ({values});"
    inserts.append(stmt)

# Preview first 5 inserts
print(f"--- Preview of INSERT statements for table '{table_name}' ---")
for i in inserts[:5]:
    print(i)

NameError: name 'df_selected' is not defined

In [23]:
# Cell 5 — (Optional) Save all INSERTs to a .sql file
if inserts:
    OUTPUT_SQL.write_text('\n'.join(inserts), encoding='utf-8')
    print(f"Successfully saved {len(inserts)} INSERT statements to: {OUTPUT_SQL}")
else:
    print("No INSERT statements were generated.")

Successfully saved 831 INSERT statements to: registration_inserts.sql
